In [27]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from tqdm import tqdm

In [28]:
class StateClassifier(nn.Module):
    def __init__(self, num_of_classes = 3):
        super().__init__()
        self.m = timm.create_model('efficientnet_b1',pretrained=True)
        self.fc = nn.Linear(self.m.classifier.out_features, num_of_classes)
    
    def forward(self, x):
        x = self.m(x)
        return self.fc(x)
    
    
class GenderClassifier(nn.Module):
    def __init__(self, num_of_classes = 2):
        super().__init__()
        self.m = timm.create_model('efficientnet_b1',pretrained=True)
        self.fc = nn.Linear(self.m.classifier.out_features, num_of_classes)
    
    def forward(self, x):
        x = self.m(x)
        return self.fc(x)
    
class AgeClassifier(nn.Module):
    def __init__(self, num_of_classes = 3):
        super().__init__()
        self.m = timm.create_model('efficientnet_b1',pretrained=True)
        self.fc = nn.Linear(self.m.classifier.out_features, num_of_classes)
    
    def forward(self, x):
        x = self.m(x)
        return self.fc(x)
       

In [29]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [30]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

# 모델과 같은 transform을 줄것!!

transform = transforms.Compose([
    Resize((260, 260), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=100
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
state_model = StateClassifier().to(device)
gender_model = GenderClassifier().to(device)
age_model = AgeClassifier().to(device)

state_model.eval()
gender_model.eval()
age_model.eval()

AgeClassifier(
  (m): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, 

In [31]:
MODEL_PATH ="saved"
state_model.load_state_dict(torch.load(os.path.join(
    MODEL_PATH, "state_model_0.00622_0.998.pt")))
gender_model.load_state_dict(torch.load(os.path.join(
    MODEL_PATH, "gender_model_0.00657_0.998.pt")))
age_model.load_state_dict(torch.load(os.path.join(
    MODEL_PATH, "age_model_0.01100_0.996.pt")))




<All keys matched successfully>

In [32]:
state_pred = []
gender_pred = []
age_pred = []
print('테스트 데이터 수:', len(loader))

## 배치 주의 -> loader의 패러미터를 버리지 않게 조정해야함
for images in tqdm(loader,'process'):
    with torch.no_grad():
        images = images.to(device)
        
        state_p = state_model(images)
        state_p = state_p.argmax(dim=-1)
        state_pred.extend(state_p.cpu().numpy())
        
        gender_p = gender_model(images)
        gender_p = gender_p.argmax(dim=-1)
        gender_pred.extend(gender_p.cpu().numpy())
        
        age_p = age_model(images)
        age_p = age_p.argmax(dim=-1)
        age_pred.extend(age_p.cpu().numpy())
        

process:   0%|          | 0/126 [00:00<?, ?it/s]

테스트 데이터 수: 126


process: 100%|██████████| 126/126 [01:40<00:00,  1.25it/s]


In [36]:
# 마스크 상태 0, 1, 2
# 성별 0, 1
# 나이 0, 1, 2

all_predictions = []
for i in range(len(state_pred)):
    
    if state_pred[i] == 0:
        
        if gender_pred[i] == 0:
            
            if age_pred[i] == 0:
                all_predictions.append(0)
            elif age_pred[i] == 1:
                all_predictions.append(1)
            elif age_pred[i] == 2:
                all_predictions.append(2)
                
        elif gender_pred[i] == 1:
            
            if age_pred[i] == 0:
                all_predictions.append(3)
            elif age_pred[i] == 1:
                all_predictions.append(4)
            elif age_pred[i] == 2:
                all_predictions.append(5)
                
    elif state_pred[i] == 1:
        
        if gender_pred[i] == 0:
            
            if age_pred[i] == 0:
                all_predictions.append(6)
            elif age_pred[i] == 1:
                all_predictions.append(7)
            elif age_pred[i] == 2:
                all_predictions.append(8)
                
        elif gender_pred[i] == 1:
            
            if age_pred[i] == 0:
                all_predictions.append(9)
            elif age_pred[i] == 1:
                all_predictions.append(10)
            elif age_pred[i] == 2:
                all_predictions.append(11)
                
                
    elif state_pred[i] == 2:
        
        if gender_pred[i] == 0:
            
            if age_pred[i] == 0:
                all_predictions.append(12)
            elif age_pred[i] == 1:
                all_predictions.append(13)
            elif age_pred[i] == 2:
                all_predictions.append(14)
                
        elif gender_pred[i] == 1:
            
            if age_pred[i] == 0:
                all_predictions.append(15)
            elif age_pred[i] == 1:
                all_predictions.append(16)
            elif age_pred[i] == 2:
                all_predictions.append(17)
                
print(len(all_predictions))

                


12600


In [37]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'test_sub.csv'), index=False)
print('test inference is done!')

test inference is done!
